In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import tensorflow as tf
import os
from tqdm import tqdm
import cv2
from PIL import Image
from joblib import Parallel, delayed
import gc
from PIL import Image as im
import random
from scipy import ndimage


2023-02-17 01:21:30.066285: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2023-02-17 01:21:30.066412: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:

#!pip install /kaggle/input/rsnapacks/dicomsdl-0.109.1-cp36-cp36m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
#import pydicom as dicom

import tensorflow as tf
from PIL import Image as im
import os
from tqdm import tqdm
import cv2
from PIL import Image
from joblib import Parallel, delayed
import gc
import random

In [3]:
with (open('/kaggle/input/processed/imgDataNew.pkl', "rb")) as openfile:
     imgData = pickle.load(openfile)
imgData=imgData[:]

2023-02-17 01:21:41.439620: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-02-17 01:21:41.443112: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/conda/lib
2023-02-17 01:21:41.443156: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-17 01:21:41.443185: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (30c80a713736): /proc/driver/nvidia/version does not exist
2023-02-17 01:21:41.447365: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operation

In [4]:
print(imgData['cancer'].value_counts())

0    42838
1    11580
Name: cancer, dtype: int64


In [5]:



train_target = np.array(imgData['cancer'])
train_features = np.array(list(imgData['img_data_process']))


gc.collect()


0

In [6]:
from tensorflow.keras.applications import * #Efficient Net included here
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import pandas as pd
from sklearn import model_selection
from tqdm import tqdm
from tensorflow.keras import optimizers
import tensorflow as tf
#Use this to check if the GPU is configured correctly
from tensorflow.python.client import device_lib

input_shape=(95,95,3)


resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)
tpu_strategy = tf.distribute.experimental.TPUStrategy(resolver)
with tpu_strategy.scope():
    conv_base = EfficientNetB0(weights="imagenet", include_top=False, \
                               input_shape=input_shape)


    model = models.Sequential()
    model.add(conv_base)
    model.add(layers.GlobalMaxPooling2D(name="gap"))
    #avoid overfitting
    model.add(layers.Dense(512, activation="relu"))
    model.add(layers.Dropout(0.3, name="dropout_out"))
    # Set NUMBER_OF_CLASSES to the number of your final predictions.
    model.add(layers.Dense(1, activation="sigmoid", name="fc_out"))
    conv_base.trainable = False

    model.compile(\
        loss="binary_crossentropy",\
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),\
        metrics=["acc"])

model.summary()



2023-02-17 01:22:01.869732: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2023-02-17 01:22:01.869799: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:30019}
2023-02-17 01:22:01.894586: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job worker -> {0 -> 10.0.0.2:8470}
2023-02-17 01:22:01.894642: I tensorflow/core/distributed_runtime/rpc/grpc_channel.cc:301] Initialize GrpcChannelCache for job localhost -> {0 -> localhost:30019}
2023-02-17 01:22:01.896387: I tensorflow/core/distributed_runtime/rpc/grpc_server_lib.cc:411] Started server with target: grpc://localhost:30019


16711680/16705208 [==============================] - 0s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb0 (Functional)  (None, 3, 3, 1280)        4049571   
_________________________________________________________________
gap (GlobalMaxPooling2D)     (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 512)               655872    
_________________________________________________________________
dropout_out (Dropout)        (None, 512)               0         
_________________________________________________________________
fc_out (Dense)               (None, 1)                 513       
Total params: 4,705,956
Trainable params: 656,385
Non-trainable params: 4,049,571
_________________________________________________________________


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#datagen = ImageDataGenerator(
#    rescale=1./255,
#    rotation_range=30,
    #shear_range=0.2,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
#    horizontal_flip=True,
#    validation_split=0.2)
#datagen.fit(train_features)


weights = {0:1, 1:4}
#model.fit(datagen.flow(train_features, train_target, batch_size=16,\
#         subset='training'),\
#         validation_data=datagen.flow(train_features, train_target,\
#         batch_size=8, subset='validation'), epochs=15)

model.fit(train_features, train_target, class_weight=weights, validation_split=0.2,batch_size=16,epochs=150)


2023-02-17 01:22:21.531379: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 1178683050 exceeds 10% of free system memory.


Epoch 1/150
2721/2721 [==============================] - 78s 24ms/step - loss: 1.2101 - acc: 0.5441 - val_loss: 0.5681 - val_acc: 0.7178
Epoch 2/150
2721/2721 [==============================] - 59s 22ms/step - loss: 1.0483 - acc: 0.5764 - val_loss: 0.6434 - val_acc: 0.6044
Epoch 3/150
2721/2721 [==============================] - 60s 22ms/step - loss: 0.9758 - acc: 0.6299 - val_loss: 0.6867 - val_acc: 0.5776
Epoch 4/150
2721/2721 [==============================] - 59s 22ms/step - loss: 0.9249 - acc: 0.6643 - val_loss: 0.5297 - val_acc: 0.7231
Epoch 5/150
2721/2721 [==============================] - 59s 22ms/step - loss: 0.8579 - acc: 0.6960 - val_loss: 0.5023 - val_acc: 0.7437
Epoch 6/150
2721/2721 [==============================] - 59s 22ms/step - loss: 0.8157 - acc: 0.7258 - val_loss: 0.4858 - val_acc: 0.7448
Epoch 7/150
2721/2721 [==============================] - 60s 22ms/step - loss: 0.7680 - acc: 0.7478 - val_loss: 0.4382 - val_acc: 0.7856
Epoch 8/150
2721/2721 [==================

In [8]:
model.save("/kaggle/working/trained_model_breast_cancer3.h5")


In [9]:
#from tensorflow import keras
#savedModel = keras.models.load_model("/kaggle/input/pre-trained-model-of-breast-cancer/trained_model_breast_cancer3.h5")



In [10]:
#test_datagen = ImageDataGenerator(
#    rescale=1./25)
#    featurewise_center=True,
#    featurewise_std_normalization=True)
#test_datagen.fit(test_features)

#test_generator = test_datagen.flow(test_features, batch_size=1)
#pred = model.predict(test_features)
#bin_pred = []
#for i in pred:
#    if i>=0.5:
#        bin_pred.append(1)
#    else:
#        bin_pred.append(0)
#bin_pred = np.array(bin_pred)


In [11]:
#import sklearn
#acc = sklearn.metrics.accuracy_score(test_target, bin_pred)
#print(acc)

#countzero=0
#countone=0
#countzerot=0
#countonet=0
#for i, j in zip(test_target, bin_pred):
#    if i==0 and j==0:
#        countzero+=1
#    if i==1 and j==1:
#        countone+=1
#    if i==1:
#        countonet+=1
#    if i==0:
#        countzerot+=1
#print(countzero/countzerot,countone/countonet)

In [12]:

def recall_m(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    possible_positives = np.sum(np.round(np.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives )
    return recall

def precision_m(y_true, y_pred):
    true_positives = np.sum(np.round(np.clip(y_true * y_pred, 0, 1)))
    predicted_positives = np.sum(np.round(np.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives )
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall))


#print(f1_m(test_target,bin_pred))